In [ ]:
#import modules
from __future__ import division
import os
import numpy as np
import skimage.io as io
import scipy.ndimage as ndi
import sympy.geometry as sp
import matplotlib.pyplot as plt

from sympy import Point3D
from skimage.morphology import disk
from scipy.spatial.distance import cdist
from skimage.segmentation import find_boundaries
%matplotlib inline

def real_space_convert(px_points, z_px_res, x_px_res, y_px_res):
    
    um_points = np.zeros_like(px_points)
    points = px_points.shape[0]
    
    for point in range(points):
        um_points[point, 0] = px_points[point, 0] * z_px_res
        um_points[point, 1] = px_points[point, 1] * x_px_res
        um_points[point, 2] = px_points[point, 2] * y_px_res
        
    return um_points

In [ ]:
# load and check dimensional order of timelapses
fpath = r'---'
fname = r'---.tif'
img4d = io.imread(os.path.join(fpath,fname))
print img4d.shape

In [ ]:
# either add new axes if reduced dimensional timelapse (i.e. single channel or single timepoint) or roll axes until desired order is acheived'''
AXIS_TO_BE_ROLLED = ---
img4d = img4d[np.newaxis]
img4d = np.rollaxis(img4d, AXIS_TO_BE_ROLLED)
print img4d.shape

In [ ]:
# pad image borders with empty pixels and create initial binary mask of cells and background subtracted images
CH = ---
PW= ---
PW2 = PW+ PW
MED_SIZE = ---

img4d_p = np.zeros((img4d.shape[0],img4d.shape[1],img4d.shape[2],img4d.shape[3]+PW2,img4d.shape[4]+PW2),dtype=np.uint8)
print img4d_p.shape
img_rdy_4d     = np.zeros_like(img4d_p[:,CH,:,:,:])
img_backsub_4d = np.zeros_like(img4d_p[:,CH,:,:,:])
mask_4d        = np.zeros_like(img4d_p[:,CH,:,:,:])

for timepoint in range(img4d.shape[0]):
    
    print 'timepoint ', timepoint
    img = img4d[timepoint,CH,:,:,:]
    print 'image shape', img.shape
    
    img_p = np.zeros((img.shape[0],img.shape[1]+PW2,img.shape[2]+PW2), dtype=np.uint8)
    print 'padded image shape', img_p.shape
    for z_slice in range(img_p.shape[0]):
        img_p[z_slice,:,:] = np.pad(img[z_slice,:,:],pad_width=PW,mode='constant')
    img4d_p[timepoint,CH,:,:,:] = img_p
    
    img_rdy = np.zeros_like(img_p)
    print 'median filtered image shape', img_rdy.shape
    for z_slice in range(img_p.shape[0]):
        img_rdy[z_slice,:,:] = ndi.median_filter(img_p[z_slice,:,:], size=MED_SIZE)
    img_rdy_4d[timepoint,:,:,:] = img_rdy
    
    stack_backsub = np.zeros_like(img_p)
    print 'background subtracted image shape', stack_backsub.shape
    stack_backsub[img_rdy>1] = img_rdy[img_rdy>1]
    img_backsub_4d[timepoint,:,:,:] = stack_backsub

    mask = np.zeros_like(img_p,dtype=np.bool)
    print 'mask shape', mask.shape
    for z_slice in range(img_p.shape[0]):
        mask[z_slice,:,:] = stack_backsub[z_slice,:,:] > stack_backsub[z_slice,:,:].mean()
    mask_4d[timepoint,:,:,:] = mask
    
io.imsave(os.path.join(fpath, fname[--:--] + '---.tif'),img_rdy_4d)
io.imsave(os.path.join(fpath, fname[--:--] + '---.tif'),img_backsub_4d)
io.imsave(os.path.join(fpath, fname[--:--] + '---.tif'),mask_4d.astype(np.uint8))

In [ ]:
# clean up binary mask and invert to obtain lumen segmentation
se = disk(---)
closed_mask_4d = np.zeros_like(mask_4d)
seg_4d = np.zeros_like(mask_4d)

for timepoint in range(mask_4d.shape[0]):
    print 'timepoint ', timepoint
    mask = mask_4d[timepoint,:,:,:]
    
    mask_rdy = np.zeros_like(mask)
    for z_slice in range(mask.shape[0]):
        #mask[z_slice,:,:] = ndi.binary_opening(mask[z_slice,:,:], structure=se, iterations=1)
        mask_rdy[z_slice,:,:] = ndi.binary_closing(mask[z_slice,:,:], structure=se, iterations=15)
    closed_mask_4d[timepoint,:,:,:] = mask_rdy

    neg_space_objs,num_NSO = ndi.label(~mask_rdy.astype(bool))
    seg_4d[timepoint,:,:,:] = neg_space_objs

lumen = np.zeros_like(mask_4d)
lumen[seg_4d>1] = seg_4d[seg_4d>1]

io.imsave(os.path.join(fpath, '---' + fname),closed_mask_4d.astype(np.uint8))
io.imsave(os.path.join(fpath, '---' + fname),seg_4d.astype(np.uint8))
io.imsave(os.path.join(fpath, '---' + fname),lumen.astype(np.uint8))

# cell below finding center of mass of Pdgfra signal & lumen boundaries

In [ ]:
# find reporter center of mass and lumen boundaries
fpath = r'---'
fname_pdg = r'---.tif'
fname_lumen = r'---.tif'

pdg = io.imread(os.path.join(fpath,fname_pdg))
print pdg.shape
lumen = io.imread(os.path.join(fpath,fname_lumen))
print lumen.shape

z  = ---
xy = ---

lumen_boundaries = np.zeros_like(lumen)
for timepoint in range(lumen.shape[0]):
    print 'timepoint', timepoint
    stack = lumen[timepoint,:,:,:]
    boundaries = find_boundaries(stack)
    lumen_boundaries[timepoint,:,:,:] = boundaries

pdg_backsub = np.copy(pdg)
pdg_centers = np.zeros((pdg.shape[0],3),dtype=np.float)
for timepoint in range(pdg.shape[0]):
    stack = pdg_backsub[timepoint,:,:,:]
    stack_center = ndi.measurements.center_of_mass(stack)
    print stack_center
    pdg_centers[timepoint,:] = stack_center

pdg_centers_um = real_space_convert(pdg_centers,z,xy,xy)
lumen_boundary = np.zeros_like(pdg_centers)
lumen_boundary_um = np.zeros_like(pdg_centers)
distances = np.zeros((pdg.shape[0],1), dtype = np.float)

for timepoint in range(lumen.shape[0]):
    print 'timepoint ', timepoint
    stack = lumen[timepoint,:,:,:]
    if stack.max() > 0:
        zB,yB,xB = np.where(lumen_boundaries[timepoint,:,:,:])
        zB = np.reshape(zB,(len(zB),1))
        yB = np.reshape(yB,(len(yB),1))
        xB = np.reshape(xB,(len(xB),1))
        coordinatesB = np.concatenate((zB,yB,xB),axis = 1)
        print coordinatesB[1,:]
        coordinatesB_um = real_space_convert(coordinatesB,z,xy,xy)
        print coordinatesB_um[1,:]
        pdg_center_um = np.reshape(pdg_centers_um[timepoint,:],(1,3))
        all_distances = cdist(pdg_center_um,coordinatesB_um)
        print all_distances.shape
        min_dist = all_distances.min(axis = 1)
        print min_dist.shape, min_dist
        distances[timepoint,:] = min_dist
    else:
        continue

np.save(os.path.join(fpath,fname_pdg[--:--]   + '---'),pdg_centers)
np.save(os.path.join(fpath,fname_pdg[--:--]   + '---'),pdg_centers_um)
np.save(os.path.join(fpath,fname_lumen[--:--] + '---'),lumen_boundaries)
np.save(os.path.join(fpath,fname_lumen[--:--] + '---'),lumen_boundary_um)
np.save(os.path.join(fpath,fname_lumen[--:--] + '---'),lumen_boundary)
np.save(os.path.join(fpath,fname_lumen[--:--] + '---'),distances)
np.save(os.path.join(fpath,fname_lumen[--:--] + '---'),coordinatesB)
np.save(os.path.join(fpath,fname_lumen[--:--] + '---'),coordinatesB_um)

# Remove false positives from lumen segmentation

In [ ]:
# remove false positives from lumen segmentation
fpath = r'---'
fname = r'---.tif'
img = io.imread(os.path.join(fpath,fname))
print img.shape

pL5voxel  = ---
img = img.astype(bool)
labeled_timelapse = np.zeros_like(img)
for timepoint in range(img.shape[0]):
    stack = img[timepoint,:,:,:]
    labels, num_labels = ndi.label(stack)
    
    sizes = ndi.measurements.sum(stack,labels,index=range(num_labels+1))
    print timepoint, num_labels, sizes
    for index,size in enumerate(sizes):
        if size < pL5voxel:
            labels[labels == index] = 0
    filt_sizes = ndi.measurements.sum(labels.astype(bool))
    print 'filt sizes', filt_sizes
    
    ### JH's alternative to filter out objects below size threshold
    ##unique_labels, unique_sizes = np.unique(label, return_counts=True)
    ##labels[np.in1d(labels.flatten(), unique_labels[unique_sizes<---]).reshape(labels.shape)] = 0
        
    labeled_timelapse[timepoint,:,:,:] = labels
    labeled_timelapse = labeled_timelapse.astype(bool)
    
io.imsave(os.path.join(fpath, fname[--:--] + '---.tif'),labeled_timelapse.astype(np.uint8))

In [ ]:
# alternative false positive removal method -- only retains the largest labeled object, not volume thresh based
se = ball(---)
ad_filt = np.zeros_like(labeled_timelapse.astype(np.uint8))
for timepoint in range(labeled_timelapse.shape[0]):
    
    stack = labeled_timelapse[timepoint,:,:,:]
    stack = ndi.binary_opening(stack,structure=se,iterations=1)
    
    labels, num_labels = ndi.label(stack)
    sizes = ndi.measurements.sum(stack,labels,index=range(num_labels+1))
    print 'number of labeled objects', num_labels
    print 'embryo size', sizes.max()
    
    for index,size in enumerate(sizes):
        if size < sizes.max():
            labels[labels == index] = 0
            
    filt_sizes = ndi.measurements.sum(labels.astype(bool))
    print 'filt sizes', filt_sizes
    print 'embryo size retained', filt_sizes.max()
    stack_filled = labels.astype(bool)
    ad_filt[timepoint,:,:,:] = stack_filled

io.imsave(os.path.join(fpath,fname[--:--] + '---.tif'),ad_filt)

In [ ]:
# calculate lumen volume
fpath = r'---'
fname = r'---.tif'
lumen_4d = io.imread(os.path.join(fpath,fname))
print lumen_4d.shape
print lumen_4d[lumen_4d[:,2]>0]

timestep = 15
start_time = 0
z_res = 2
xy_res = 0.2306294
lumen_4d = lumen_4d.astype(bool)
lumen_size = np.zeros((lumen_4d.shape[0],3),dtype=np.uint16)

for timepoint in range(lumen_4d.shape[0]):
    lumen = lumen_4d[timepoint,:,:,:]
    size = np.sum(lumen)
    size_pL = size*z_res*xy_res*xy_res*0.001
    print size
    lumen_size[timepoint,0] = timepoint
    lumen_size[timepoint,1] = timepoint*timestep + start_time
    lumen_size[timepoint,2] = size_pL
print lumen_size

np.save(os.path.join(fpath, fname[--:--] + '---.npy'),lumen_size)

In [ ]:
# sum reporter expression
fpath = r'---'
fname = r'---'
pdg = io.imread(os.path.join(fpath,fname))
print pdg.shape

pdg_exp = np.zeros((pdg.shape[0],1),dtype=np.float)
for timepoint in range(pdg.shape[0]):
    
    stack = pdg[timepoint,:,:,:]
    exp_size = np.sum(stack.astype(bool))
    stack_exp = np.sum(stack)/exp_size
    
    print stack_exp
    pdg_exp[timepoint,:] = stack_exp
    
np.save(os.path.join(fpath, fname[--:--] + '---.npy'),pdg_exp)